In [56]:
import docx2txt
import string

In [79]:
tmp_lines = []
def isAllCaps(line):
    for ch in line:
        if(ch.islower()):
            return False
    return True
def isActionWithOneLinebreak(line):
    if line == "":
        return True
    if line[0] =='(' or line[-1] ==')':
        return False
    if not isName(line):
        return True
        
    return False
def checkNext3Lines(i,lines):
    return (isActionWithOneLinebreak(lines[i+1]) and 
    isActionWithOneLinebreak(lines[i+2]) and
    isActionWithOneLinebreak(lines[i+3]))
def checkPrevious3Lines(i,lines):
    return (isActionWithOneLinebreak(lines[i-1]) and 
    isActionWithOneLinebreak(lines[i-2]) and
    isActionWithOneLinebreak(lines[i-3]) and
    not isName(lines[i-4]))

def isSceneChanger(line):#deteck scene changer lines
    sc_list = ["EXT.","INT.","DECK "," - ","ROOM","ELEVATOR","ON THE "]
    for sc in sc_list:
        if(sc in line):
            return True
    return False
def checkPreviousNonemptyLine(i,lines):
    print(lines[i],"returns")
    while(lines[i-1] == ""):
        i -= 1
    i -= 1
    if i ==0:
        return True
    #print(i)
    
    print(lines[i])
    if lines[i][0] =='(' and lines[i][-1] ==')':
        return True
   
    print('False')
    return False
    
def isAction(i,lines):
    debug_line = 6900
    #print("i-1,i-3=>",lines[i-1] , lines[i-2] , lines[i-3])
    if debug_line+200>i>debug_line:
        print("try.. ",i,lines[i])
    if(lines[i] == "" or i<4):
        return False
    if(i+3>=len(lines)):
        return False
    if( debug_line+200>i>debug_line ):
        print(lines[i])
    #if("--" in lines[i]):
     #   return False
    if lines[i][-1] == '?':
        return False
    if lines[i] == ' ' or lines[i] == '\t':
        return True
    if('(' is lines[i][0] or ')' is lines[i][-1]):
        return False
    if(isAllCaps(lines[i])):
        return False
    if isName(lines[i-1]) or isName(lines[i-2]) or isName(lines[i-3]):
        return False
    if( debug_line+200>i>debug_line ):
        print(lines[i])
    if(checkPrevious3Lines(i,lines)):
        if(checkNext3Lines(i,lines)):
    #        if(len(lines[i])<3):
    #            print('-',len(lines[i]),'-',lines[i],'-')
            if(not lines[i][1].isupper()):
                if debug_line+200>i>debug_line:
                    print("still try.. ",i,lines[i])
                if(checkPreviousNonemptyLine(i,lines)):
                    return False
                if(lines[i-4] == ""):
                    return True
                return True
    if( debug_line+200>i>debug_line ):
        print(lines[i])
    if isSceneChanger(lines[i-4]):
        return True
    if lines[i][0].islower():
        return False
    words_in_line = lines[i].split()
    for word in words_in_line:
        if(isName(word)):
            return True
        
    return False
def isName(name):
    if not isAllCaps(name):
        return False
    if isSceneChanger(name):
        return False
    if len(name)<3:
        return False
    return True
#todo:checks whether the file has errors or not
def errorInFile(lines):
    for line in lines:
        if(bool("(" in line) != bool(")" in line)):
            pass
            #print("(" in line , ")" in line)
            #print(line)
            #return True
    return False

In [83]:
MOVIE_NAME = "As above so below"
DOCX_NAME = "As-Above-So-Below_Dowdle's-2013-15-02" #without extension
my_text = docx2txt.process(DOCX_NAME +".docx")
lines = my_text.split('\n')
out = open(DOCX_NAME + "_out.txt","w")
txtfile = open(DOCX_NAME + "_modified.txt","w")
orgfile = open(DOCX_NAME+ "_original.txt","w")
print("Movie:",MOVIE_NAME,"Screenplay:",DOCX_NAME)
discovery = 1

if(errorInFile(lines)):
    print("ERROR IN FILE")
    exit(1)
tmp_lines = []
print('printing original file to txt..')
for i,line in enumerate(lines):
    orgfile.write(lines[i]+'\n')
#remove tabs,(MORE),pagenumbers from text    
print("Modifing the file...")
for i,line in enumerate(lines):
    lines[i] = lines[i].replace('(MORE)','')#replace all (MORE)
    lines[i] = lines[i].replace('EXT:','EXT.') # default is 'EXT.', not 'EXT:'
    lines[i] = lines[i].replace("cont'd","CONT'D")
    lines[i] = lines[i].replace("’","'")
    lines[i] = lines[i].replace('MNightFans.com','')
    if(lines[i]==''):
        continue
    lines[i] = lines[i].replace('\t',' ')#replace all tab characters with space
    if('BACK TO' in lines[i]):#erase those
        lines[i] = ''
    if('CLOSE ON' in lines[i]):#direction for camera
        lines[i] = ''
    if('DISSOLVE TO' in lines[i] or 'CUT ' in lines[i]):
        lines[i] = ''
    if('ONSCREEN' in lines[i] or 'FADE' in lines[i]):
        lines[i] = ''
    if('TO CAMERA' in lines[i]):# or 'to camera' in lines[i]
        lines[i] = ''
    if("(CONT'D)" in lines[i] and lines[i][-8:] != "(CONT'D)"):
        #print(lines[i])
        cont_splitted = lines[i].split("(CONT'D)")
        lines[i] = cont_splitted[0]
        lines.insert(i+1,"")
        lines.insert(i+2,cont_splitted[1][1:])
        #fix?
for i,line in enumerate(lines):
    #print(lines[i])
    if(lines[i]==''):
        continue
    while(lines[i][0] == ' '):#remove space if it's the first character
        #print("tab:",lines[i])
        lines[i] = lines[i][1:]
        #print("aft:",lines[i])
    if(2<=len(lines[i])<=5 and lines[i][-1] == '.' and  lines[i][:-1].isdigit()):#replace pagenumbers
        #print("page:",lines[i])
        lines[i] = ""
        continue
    if(1<=len(lines[i])<=4 and lines[i].isdigit()):#replace pagenumbers
        #print("page:",lines[i])
        lines[i] = ""
        continue    
    if(lines[i] != "" and ')' in lines[i] and '(' not in lines[i]):#find parantheticals that occupy more than oneline,merge them
        open_id = i
        #print("=>",lines[i-1],lines[i])
        while('(' not in lines[open_id]):
            open_id -= 1
        new_line = lines[open_id]+" "+lines[i]
        #print(new_line)
        del lines[open_id:i+1]
        lines.insert(open_id,new_line)
    lines[i] = lines[i].replace('(voice over)','(V.O.)')
    if(lines[i][0:4] == "(O.S" or lines[i][0:4] == "(V.O"):# if O.S or V.O is the beginning of a line, append it to its speaker
        lines[i-2] +=" "+ lines[i]
        lines[i] = lines[i-2]
    
        
test = 988
#print("%".join(lines[test-10:test+10]))
print("Remove unnecassary action(direction)s")

for i,line in enumerate(lines):
    #remove action(direction)'s and 3 '\n' after them
    if(isAction(i,lines)):
    #    print(i,lines[i])
        continue
        #del lines[i:i+4]
        
    else:
        tmp_lines.append(lines[i])
    #if i > 1000:
     #   break
lines = tmp_lines
#print modified file to check
for i,line in enumerate(lines):
    txtfile.write(line+"\n")
print("Parsing..")
to_be_printed = []
for i,line in enumerate(lines):
    if('(' in line):
        if(isAllCaps(line)):
            continue
        if(line[0]=='(' and line[-1]==')' and line[1:10] == 'copyright'):
            continue
        if("CONT'D" not in line and "O.S." not in line and "V.O" not in line and "MORE" not in line):
            pre_emotion_empty_count = 0
            pre_step = 0
            pre_name_count = 0
            result = []
        
            while(pre_name_count < 1 or len(lines[i - pre_step]) < 3 or  not  isAllCaps(lines[i-pre_step])):
                if isAllCaps(lines[i-pre_step]) and len(lines[i-pre_step]) > 0:
                    pre_name_count += 1
                pre_step += 1
            if(isSceneChanger(lines[i-pre_step])):
                result.append("")
                result.append("")
                result.append("")
                pre_step-=1
                while(lines[i-pre_step] == ""):
                    pre_step-=1
                
            post_emotion_empty_count = 0
            post_step = 0
            post_name_count = 0

            while(post_name_count < 2 or len(lines[i + post_step]) < 3 or  not  isAllCaps(lines[i+post_step]) ):
                if isSceneChanger(lines[i+post_step]) :
                    break
                if isAllCaps(lines[i+post_step]) and len(lines[i+post_step]) > 0:
                    post_name_count += 1
                post_step += 1
            while isSceneChanger(lines[i+post_step]):
                post_step -=1
            
            #print(lines[i-pre_step:i+post_step])
            #print(pre_step,post_step)
            printed_lines = lines[i-pre_step:i+post_step]
            
            for line_in_range in lines[i-pre_step:i+post_step]:
                if(line_in_range!="" and not line_in_range[:-1].isnumeric()):
                    if(line_in_range!="(O.S.)"):
                        if(len(result)%3 == 1 and result[-1][0] != '(' and line_in_range[0]!='('):
                            result.append("")
                        result.append(line_in_range)
            name_utterance = []
            tmp_utterance = "%%%"
            for x in result:
                if(isName(x)):
                    if(tmp_utterance != "%%%"):
                        name_utterance.append(tmp_utterance)
                    tmp_utterance = ""
                    name_utterance.append(x)
                else:
                    if(x!=""):
                        tmp_utterance +=" "
                    tmp_utterance +=""+ x
                
            if(tmp_utterance != ""):
                name_utterance.append(tmp_utterance)
            #print("=".join(name_utterance))
            #out.write("\t".join(name_utterance)+"\n")
            while(len(name_utterance)<8):
                name_utterance.append("")
            to_be_printed.append(name_utterance + [MOVIE_NAME])
            #print("+".join(result))
            #out.write("\t".join(result)+"\n")
            #print(discovery,i)
            discovery+=1
            #if(discovery>50):
             #   break
    elif(line.count("(")>2):
        print("Line {0} has more than one (",i)
for i,l in enumerate(to_be_printed):
    if(i == len(to_be_printed)-1 or not (l[0]==to_be_printed[i+1][0] and l[1]==to_be_printed[i+1][1])):
        out.write("\t".join(l)+"\n")
            
out.close()
txtfile.close()
orgfile.close()
print("Done")


Movie: As above so below Screenplay: As-Above-So-Below_Dowdle's-2013-15-02
printing original file to txt..
Modifing the file...
Remove unnecassary action(direction)s
Written by returns
John Erick Dowdle returns
Written by
False
Drew Dowdle returns
John Erick Dowdle
False
Feb. 15, 2013 returns
Drew Dowdle
False
Camera powers up, jostles around until it settles on -- returns
INT. IRANIAN BUS - AFTERNOON
False
The floor of a third world bus. Middle Eastern legs, traditional Arabian dresses to the floor, homemade bags, animals everywhere. A chaotic scene with men, women, children all shouting in Farsi. Brief glimpses of a mountainous landscape through the shaky windows. This bus could really use some new shocks and a better muffler. returns
Camera powers up, jostles around until it settles on --
False
The camera rises, turns around revealing its operator, a WOMAN wearing a hajib (a traditional Arabian headdress). She slumps low in her seat for privacy, looks around cautiously, and carefull

In [82]:
print(' '.islower())

False
